# Installing Packages

In [ ]:
https://www.kaggle.com/code/fabiendaniel/predicting-flight-delays-tutorial/notebook

In [1]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6
!pip install python-scipy

     |████████████████████████████████| 1.8MB 10.4MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
  Attempting uninstall: snowflake-snowpark-python
    Found existing installation: snowflake-snowpark-python 1.23.0
    Uninstalling snowflake-snowpark-python-1.23.0:
      Successfully uninstalled snowflake-snowpark-python-1.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.1.8 requires cloudpickle==2.2.1, but you have cloudpickle 2.0.0 which is inco

In [2]:
!pip install basemap
#!pip install libgeos-3.5.0
#!pip install libgeos-dev
#!pip install https://github.com/matplotlib/basemap/archive/master.zip

# Importing packages

In [5]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

# Reading all datasets from SF

In [6]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [9]:
# Define the table name
table_name = 'FLIGHTS'

# Execute SQL query to retrieve the data from the specified table
df = my_session.sql("select * from {}".format(table_name))

# Convert the Snowflake DataFrame to a pandas DataFrame for easier manipulation and analysis
df = df.to_pandas()

# Display the data
df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,7,19,7,UA,1537,N37427,LAS,EWR,1201,1202.0,1.0,13.0,1215.0,299.0,307.0,278.0,2227,1953.0,16.0,2000,2009.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5819075,2015,7,19,7,OO,5531,N929SW,SMF,DEN,1201,1203.0,2.0,14.0,1217.0,145.0,140.0,122.0,909,1519.0,4.0,1526,1523.0,-3.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5819076,2015,7,19,7,DL,1424,N686DA,MCO,DTW,1202,1153.0,-9.0,26.0,1219.0,158.0,173.0,142.0,957,1441.0,5.0,1440,1446.0,6.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5819077,2015,7,19,7,EV,4535,N34110,EWR,AVP,1202,1202.0,0.0,17.0,1219.0,56.0,47.0,24.0,93,1243.0,6.0,1258,1249.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN


In [10]:
# Define the table name
table_name = 'AIRLINES'

# Execute SQL query to retrieve the data from the specified table
airlines_df = my_session.sql("select * from {}".format(table_name))

# Convert the Snowflake DataFrame to a pandas DataFrame for easier manipulation and analysis
airlines_df = airlines_df.to_pandas()

# Display the data
airlines_df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [11]:
# Define the table name
table_name = 'AIRPORTS'

# Execute SQL query to retrieve the data from the specified table
airports_df = my_session.sql("select * from {}".format(table_name))

# Convert the Snowflake DataFrame to a pandas DataFrame for easier manipulation and analysis
airports_df = airports_df.to_pandas()

# Display the data
airports_df

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023
